In [ ]:
%pip install ultralytics opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install gdown



   ---------------------------------------- 0/4 [soupsieve]
   ---------------------------------------- 0/4 [soupsieve]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   ------------------------------

In [ ]:
from ultralytics import YOLO
import gdown
import cv2
import numpy as np
from collections import defaultdict

In [4]:
file_id = "18FyNqEWelu83JAZ5_8TrNlzxQQb7KUjF"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="video_input.mp4", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=18FyNqEWelu83JAZ5_8TrNlzxQQb7KUjF
To: d:\Projects\Car Detection With YOLO\video_input.mp4
100%|██████████| 1.97M/1.97M [00:41<00:00, 47.9kB/s]


'video_input.mp4'

In [5]:
model = YOLO("yolo11n.pt") 
video_path = "video_input.mp4" 

cap = cv2.VideoCapture(video_path) 

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracking.mp4', fourcc, fps, (width, height))


100%|██████████| 5.35M/5.35M [00:26<00:00, 215kB/s] 


In [6]:
track_history = defaultdict(lambda: [])
car_ids = set() 

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    results = model.track(frame, persist=True, classes=[2])  
    result = results[0]

    if result.boxes and result.boxes.is_track:
        boxes = result.boxes.xywh.cpu().numpy()
        ids = result.boxes.id.int().cpu().tolist()

        frame = result.plot()

        for box, track_id in zip(boxes, ids):
            x, y, w, h = box
            car_ids.add(track_id)

            track = track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 255), thickness=2)

        cv2.putText(frame, f"Total Cars Detected: {len(car_ids)}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    out.write(frame)

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 82.5 kB/s eta 0:00:003

requirements: AutoUpdate success  21.2s, installed 1 package: ['lap>=0.5.12']
requirements:  Restart runtime or rerun command for updates to take effect


0: 384x640 1 car, 365.7ms
Speed: 21.2ms preprocess, 365.7ms inference, 17.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 228.2ms
Speed: 4.9ms preprocess, 228.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 142.2ms
Speed: 4.1ms preprocess, 142.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 242.1ms
Speed: 4.0ms preprocess, 242.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 151.1ms
Speed: 3.0ms preprocess, 151.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 195.1ms
Speed: 4.8ms preproc

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video tracking selesai, lihat hasilnya di file output_tracking.mp4")

Video tracking selesai, lihat hasilnya di file output_tracking.mp4
